## Run this block of codes to install the required IBM packages

https://qiskit-community.github.io/qiskit-optimization/index.html

https://qiskit-community.github.io/qiskit-optimization/tutorials/04_grover_optimizer.html

In [ ]:
!pip install qiskit
!pip install qiskit_ibm_runtime
!pip install pylatexenc
!pip install qiskit-optimization
!pip install qiskit_algorithms
!pip install tutorial_magics
!pip install qiskit_aer

## Run this block of codes to connect to IBM server
There is no need to run this block of code for the simulator

In [ ]:
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit_aer import AerSimulator

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService(channel="ibm_quantum", 
                               token="07bfd9f7167962a67efe94379d7c466db6655eb4b8dbeb2846891b29d76eb12672a900e591fe0e04a67a0fa6e01e0923545b07282473e979f2dc9ba6f5dea66e")

## Run this block of codes to load required packages

In [38]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram
from typing import List, Tuple

try:
    import cplex
    from cplex.exceptions import CplexError
except:
    print("Warning: Cplex not found.")
import math
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit_optimization.algorithms import SolutionSample, OptimizationResultStatus

from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model

from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit.circuit.library import RealAmplitudes

from qiskit_optimization import QuadraticProgram

## Job shop problem definition

In [39]:
model = Model(name='job_shop')

num_jobs = 2
num_machines = 2 + 1
num_time_slots = 4

J = range(1, num_jobs+1)  # jobs
M = range(1, num_machines + 1)   # machines
T = range(1, num_time_slots+1)  # time slots

# Define constants
w = {1: 10, 2: 30}  # j: tardiness weights
E = {(1, 1): 5, (1, 2): 10, (1, 3): 0, 
     (2, 1): 0, (2, 2): 0, (2, 3): 0}  # (j, m): earliness weights
d = {1: 4, 2:4}  #  j: due dates
p = {(1, 1): 2, (1, 2): 1, (1, 3): 0, 
     (2, 1): 0, (2, 2): 1, (2, 3): 0}  # (j, m): processing times
P = {(1, 1): 3,  (1, 2): 1, (1, 3): 0, 
     (2, 1): 0,  (2, 2): 1, (2, 3): 0}  # cumulative processing times backward
Pp = {(1, 1): 1,  (1, 2): 0, (1, 3): 0, 
      (2, 1): 0,  (2, 2): 0, (2, 3): 0}  # cumulative processing times backward
mp = {(1, 2): 1, (1, 3): 2, 
    (2, 3): 2} #  (j, m): previous machine

i_value = 1
sumE = {i: sum(value for key, value in E.items() if key[0] == i) for i in set(key[0] for key in E)}

h = [0, 0, 0, 0, 0]

## Create Variables for the problem

In [40]:
variables = {}

for j in J:
    for m in M:
        for t in T:
            var_name = f"y_{j}_{m}_{t}"
            variables[(j,m, t)] = model.binary_var(name=var_name)

print("number of variables: ", len(variables))

number of variables:  24


## Objective function: W E/T Cost

In [41]:
objective = 0
for j in J:
    objective = objective - d[j]*w[j]
    for t in range(d[j], num_time_slots+1):
        objective = objective + ((w[j]+sumE[j])*t)*variables[(j, num_machines, t)]
    tmp = 0
    for tp in range(1, num_time_slots+1):
        for m in range(1, num_machines):
            tmp = tmp - E[j,m]*(tp + Pp[j,m])*variables[(j, m, tp)]
    objective = objective + tmp

h[0] = objective
objective

docplex.mp.LinearExpr(-10y_1_1_1-15y_1_1_2-20y_1_1_3-25y_1_1_4-10y_1_2_1-20y_1_2_2-30y_1_2_3-40y_1_2_4+100y_1_3_4+120y_2_3_4-160)

## Constraint 1: No − overlap

In [42]:
h1 = 0
for m in range(1, num_machines):
    for j in J:
        for t in T:
            for jp in J:
                if j!= jp:
                    for tp in range(t, t+ p[j,m] - 1 + 1):
                        if tp <= num_time_slots:
                            h1 = h1 + variables[(j, m, t)]*variables[(jp, m, tp)]
h[1] = h1
h1

docplex.mp.quad.QuadExpr(y_1_1_1*y_2_1_1+y_1_1_1*y_2_1_2+y_1_1_2*y_2_1_2+y_1_1_2*y_2_1_3+y_1_1_3*y_2_1_3+y_1_1_3*y_2_1_4+y_1_1_4*y_2_1_4+2y_1_2_1*y_2_2_1+2y_1_2_2*y_2_2_2+2y_1_2_3*y_2_2_3+2y_1_2_4*y_2_2_4)

## Constraint 2: Precedence

In [43]:
h2 = 0
for j in J:
    for m in range(2, num_machines):
        if (j, m) in mp:
            for t in T:
                for tp in range(1, t + p[j,m] - 1 + 1):
                    if tp <= num_time_slots:
                        h2 = h2 + variables[(j, mp[j,m], t)]*variables[(j, m, tp)]
h[2] = h2

for j in J:
    m = num_machines
    for t in range(d[j], num_time_slots+1):
        for tp in range(1, t - 1 + 1):
            h2 = h2 + variables[(j, mp[j,m], t)]*variables[(j, m, tp)]
h[2] = h2

h2

docplex.mp.quad.QuadExpr(y_1_1_1*y_1_2_1+y_1_1_2*y_1_2_1+y_1_1_2*y_1_2_2+y_1_1_3*y_1_2_1+y_1_1_3*y_1_2_2+y_1_1_3*y_1_2_3+y_1_1_4*y_1_2_1+y_1_1_4*y_1_2_2+y_1_1_4*y_1_2_3+y_1_1_4*y_1_2_4+y_1_2_4*y_1_3_1+y_1_2_4*y_1_3_2+y_1_2_4*y_1_3_3+y_2_2_4*y_2_3_1+y_2_2_4*y_2_3_2+y_2_2_4*y_2_3_3)

## Constraint 3: Operation once

In [44]:
h3 = 0
for j in J:
    for m in range(1, num_machines):
        if p[j,m] > 0:
            tmp = 0
            for t in T:
                tmp = tmp + variables[(j, m, t)]
            h3 = h3 + (tmp - 1)*(tmp - 1)
h[3] = h3
h3

docplex.mp.quad.QuadExpr(y_1_1_1^2+2y_1_1_1*y_1_1_2+2y_1_1_1*y_1_1_3+2y_1_1_1*y_1_1_4+y_1_1_2^2+2y_1_1_2*y_1_1_3+2y_1_1_2*y_1_1_4+y_1_1_3^2+2y_1_1_3*y_1_1_4+y_1_1_4^2+y_1_2_1^2+2y_1_2_1*y_1_2_2+2y_1_2_1*y_1_2_3+2y_1_2_1*y_1_2_4+y_1_2_2^2+2y_1_2_2*y_1_2_3+2y_1_2_2*y_1_2_4+y_1_2_3^2+2y_1_2_3*y_1_2_4+y_1_2_4^2+y_2_2_1^2+2y_2_2_1*y_2_2_2+2y_2_2_1*y_2_2_3+2y_2_2_1*y_2_2_4+y_2_2_2^2+2y_2_2_2*y_2_2_3+2y_2_2_2*y_2_2_4+y_2_2_3^2+2y_2_2_3*y_2_2_4+y_2_2_4^2-2y_1_1_1-2y_1_1_2-2y_1_1_3-2y_1_1_4-2y_1_2_1-2y_1_2_2-2y_1_2_3-2y_1_2_4-2y_2_2_1-2y_2_2_2-2y_2_2_3-2y_2_2_4+3)

## Constraint 4: Tardy − once

In [45]:
h4 = 0
for j in J:
    tmp = 0
    for t in range(d[j], num_time_slots+1):
        tmp = tmp + variables[(j, num_machines, t)]
    h4 = h4 + (tmp - 1)*(tmp - 1)
h[4] = h4
h4

docplex.mp.quad.QuadExpr(y_1_3_4^2+y_2_3_4^2-2y_1_3_4-2y_2_3_4+2)

## Create the QUBO model and solve it using MinimumEigenOptimizer

In [46]:
alpha = [1, 1000, 1000, 1000, 1000]

model.minimize(alpha[0]*h[0]+alpha[1]*h[1]+alpha[2]*h[2]+alpha[3]*h[3]+alpha[4]*h[4])
qp = from_docplex_mp(model)
print(qp.prettyprint())

Problem name: job_shop

Minimize
  1000*y_1_1_1^2 + 2000*y_1_1_1*y_1_1_2 + 2000*y_1_1_1*y_1_1_3
  + 2000*y_1_1_1*y_1_1_4 + 1000*y_1_1_1*y_1_2_1 + 1000*y_1_1_1*y_2_1_1
  + 1000*y_1_1_1*y_2_1_2 + 1000*y_1_1_2^2 + 2000*y_1_1_2*y_1_1_3
  + 2000*y_1_1_2*y_1_1_4 + 1000*y_1_1_2*y_1_2_1 + 1000*y_1_1_2*y_1_2_2
  + 1000*y_1_1_2*y_2_1_2 + 1000*y_1_1_2*y_2_1_3 + 1000*y_1_1_3^2
  + 2000*y_1_1_3*y_1_1_4 + 1000*y_1_1_3*y_1_2_1 + 1000*y_1_1_3*y_1_2_2
  + 1000*y_1_1_3*y_1_2_3 + 1000*y_1_1_3*y_2_1_3 + 1000*y_1_1_3*y_2_1_4
  + 1000*y_1_1_4^2 + 1000*y_1_1_4*y_1_2_1 + 1000*y_1_1_4*y_1_2_2
  + 1000*y_1_1_4*y_1_2_3 + 1000*y_1_1_4*y_1_2_4 + 1000*y_1_1_4*y_2_1_4
  + 1000*y_1_2_1^2 + 2000*y_1_2_1*y_1_2_2 + 2000*y_1_2_1*y_1_2_3
  + 2000*y_1_2_1*y_1_2_4 + 2000*y_1_2_1*y_2_2_1 + 1000*y_1_2_2^2
  + 2000*y_1_2_2*y_1_2_3 + 2000*y_1_2_2*y_1_2_4 + 2000*y_1_2_2*y_2_2_2
  + 1000*y_1_2_3^2 + 2000*y_1_2_3*y_1_2_4 + 2000*y_1_2_3*y_2_2_3
  + 1000*y_1_2_4^2 + 1000*y_1_2_4*y_1_3_1 + 1000*y_1_2_4*y_1_3_2
  + 1000*y_1_2_4*y_1_3_

In [48]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print(exact_result.prettyprint())

for var, value in exact_result.variables_dict.items():
    if value > 0:
        print(f'{var}: {value}')

objective function value: 0.0
variable values: y_1_1_1=0.0, y_1_1_2=0.0, y_1_1_3=1.0, y_1_1_4=0.0, y_1_2_1=0.0, y_1_2_2=0.0, y_1_2_3=0.0, y_1_2_4=1.0, y_1_3_1=0.0, y_1_3_2=0.0, y_1_3_3=0.0, y_1_3_4=1.0, y_2_1_1=1.0, y_2_1_2=0.0, y_2_1_3=0.0, y_2_1_4=0.0, y_2_2_1=0.0, y_2_2_2=0.0, y_2_2_3=1.0, y_2_2_4=0.0, y_2_3_1=0.0, y_2_3_2=0.0, y_2_3_3=1.0, y_2_3_4=1.0
status: SUCCESS
y_1_1_3: 1.0
y_1_2_4: 1.0
y_1_3_4: 1.0
y_2_1_1: 1.0
y_2_2_3: 1.0
y_2_3_3: 1.0
y_2_3_4: 1.0


In [25]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
qaoa_result = qaoa.solve(qp)
print(qaoa_result.prettyprint())

# print("variable order:", [var.name for var in qaoa_result.variables])
# for s in qaoa_result.samples:
#     print(s)

def get_filtered_samples(
    samples: List[SolutionSample],
    threshold: float = 0,
    allowed_status: Tuple[OptimizationResultStatus] = (OptimizationResultStatus.SUCCESS,),
):
    res = []
    for s in samples:
        if s.status in allowed_status and s.probability > threshold:
            res.append(s)

    return res

filtered_samples = get_filtered_samples(
    qaoa_result.samples, threshold=0.005, allowed_status=(OptimizationResultStatus.SUCCESS,)
)
# for s in filtered_samples:
#     print(s)

fvals = [s.fval for s in qaoa_result.samples]
probabilities = [s.probability for s in qaoa_result.samples]

samples_for_plot = {
    " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x)): s.probability
    for s in filtered_samples
}

# plot_histogram(samples_for_plot)

print(samples_for_plot)


objective function value: -140.0
variable values: y_1_1_1=1.0, y_1_1_2=0.0, y_1_1_3=0.0, y_1_1_4=0.0, y_1_2_1=0.0, y_1_2_2=1.0, y_1_2_3=1.0, y_1_2_4=1.0, y_1_3_1=0.0, y_1_3_2=0.0, y_1_3_3=0.0, y_1_3_4=0.0
status: SUCCESS
{'y_1_1_1=1 y_1_1_2=1 y_1_1_3=0 y_1_1_4=1 y_1_2_1=0 y_1_2_2=0 y_1_2_3=0 y_1_2_4=0 y_1_3_1=0 y_1_3_2=0 y_1_3_3=0 y_1_3_4=0': 0.0197691805997289, 'y_1_1_1=1 y_1_1_2=1 y_1_1_3=0 y_1_1_4=1 y_1_2_1=0 y_1_2_2=0 y_1_2_3=0 y_1_2_4=0 y_1_3_1=1 y_1_3_2=0 y_1_3_3=0 y_1_3_4=0': 0.008771365257339, 'y_1_1_1=1 y_1_1_2=1 y_1_1_3=0 y_1_1_4=1 y_1_2_1=0 y_1_2_2=0 y_1_2_3=0 y_1_2_4=0 y_1_3_1=0 y_1_3_2=1 y_1_3_3=0 y_1_3_4=0': 0.008771365257339, 'y_1_1_1=1 y_1_1_2=1 y_1_1_3=0 y_1_1_4=1 y_1_2_1=0 y_1_2_2=0 y_1_2_3=0 y_1_2_4=0 y_1_3_1=0 y_1_3_2=0 y_1_3_3=1 y_1_3_4=0': 0.008771365257339, 'y_1_1_1=0 y_1_1_2=1 y_1_1_3=0 y_1_1_4=1 y_1_2_1=0 y_1_2_2=0 y_1_2_3=0 y_1_2_4=0 y_1_3_1=0 y_1_3_2=0 y_1_3_3=0 y_1_3_4=0': 0.0405783369686376, 'y_1_1_1=0 y_1_1_2=1 y_1_1_3=0 y_1_1_4=1 y_1_2_1=0 y_1_2_2=0 y_1_2

## Solve the model using Grover Adaptive Search

In [ ]:
grover_optimizer = GroverOptimizer(len(variables)*2, num_iterations=20, sampler=Sampler())
results = grover_optimizer.solve(qp)
print(results.prettyprint())